# TODO:
* **Criar uma funcao que colete os links de todos os imoveis a venda da imobiliaria ✓**

* **Criar um dicionario com as informacoes de: Preco ✓**

# Estudo da página

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from pandas import DataFrame

In [36]:
driver = webdriver.Firefox(executable_path='/home/douglas/PycharmProjects/scraperImperium/geckodriver')

/tmp/ipykernel_4492/1040251504.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='/home/douglas/PycharmProjects/scraperImperium/geckodriver')


In [37]:
driver.get('https://www.imperiumimoveis.com.br/imoveis/venda-guarapuava')

**Obtendo quantidade de imóveis**

In [38]:
qtd = driver.find_element(By.CLASS_NAME, 'list__counter').text
qtd = int(qtd.split()[0])

**O tamanho inicial já conta com 12 anúncios. Cada seção/load da página adiciona 3960 pixels/12 anúncios. Existem 413 anúncios**

In [39]:
def rolar_pag():
    # Impede que o site "emperre"
    driver.execute_script('window.scrollBy(0, -500)')
    # Elemento que faz com que o site carregue mais anúncios
    elemento = driver.find_element(By.XPATH, '/html/body/main/div/section/div[2]/div[1]/div[3]/div[2]')
    #Move para o elemento que carrega a página
    driver.execute_script("arguments[0].scrollIntoView();", elemento)

**Carregando todos os anúncios**

In [40]:
maximo = 0
while True:
    rolar_pag()
    sleep(1.5)
    aux = driver.execute_script('return document.body.scrollHeight')
    if maximo < aux:
        maximo = aux
    else:
        break

O elemento ```<div class="jetgrid list__container" data-template="">``` é pai de todos os elementos de anúncios (```<div class="jetgrid__col--24 jetgrid__col--sm-12 jetgrid__col--xs-24">```)

XPath pai:
```/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]```

XPath 1º anúncio:
```/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[1]```

XPath 2º anúncio:
```/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[2]```

# Resolução atividade

**Coletar Links**

In [41]:
def coletar_links():
    links = []
    for i in range(1, qtd+1):
        link = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a').get_attribute('href')
        links.append(link)
    return links

In [44]:
coletar_links()

['https://www.imperiumimoveis.com.br/imovel/terreno-alto-cascavel-guarapuava-3351m2-venda-rs17000000?id=9970&cod=530809970&search=5886662',
 'https://www.imperiumimoveis.com.br/imovel/terreno-vila-carli-guarapuava-20000m2-venda-rs6000000?id=9970&cod=467209970&search=5886662',
 'https://www.imperiumimoveis.com.br/imovel/sobrado-santa-cruz-guarapuava-com-garagem-600m2-venda-rs5500000?id=9970&cod=413009970&search=5886662',
 'https://www.imperiumimoveis.com.br/imovel/terreno-centro-guarapuava-2100m2-venda-rs4000000?id=9970&cod=478609970&search=5886662',
 'https://www.imperiumimoveis.com.br/imovel/imovel-comercial-centro-guarapuava-646m2-venda-rs3700000?id=9970&cod=432109970&search=5886662',
 'https://www.imperiumimoveis.com.br/imovel/imovel-comercial-centro-guarapuava-venda-rs3600000?id=9970&cod=426509970&search=5886662',
 'https://www.imperiumimoveis.com.br/imovel/terreno-alto-da-xv-guarapuava-venda-rs3400000?id=9970&cod=413209970&search=5886662',
 'https://www.imperiumimoveis.com.br/imov

**Dicionário de Preços**

In [45]:
def tratar_preco_div(entrada : str) -> float:
    '''Para preços com tag final <div>'''
    x = entrada.split()[1]
    x = x.replace('.', '').replace(',', '.')
    return float(x)

In [53]:
# 'VENDA:\n25.000,00'
def tratar_preco_p(entrada : str) -> float:
    '''Para preços com tag final <p>'''
    x = entrada.replace('VENDA:\n', '')
    x = x.replace('.', '').replace(',', '.')
    return float(x)

In [54]:
dicio = {}
for i in range(1, qtd+1):
    controlador = True
    # Número de matrícula do imóvel? É o que fazia mais sentido para ser uma chave de dicionário
    num = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/p[1]').text
    try:
        preco = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/div').text
        preco = tratar_preco_div(preco)
        controlador = False
    except:
        pass

    if controlador:
        try:
            preco = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/p[4]').text
            preco = tratar_preco_p(preco)
            controlador = False
        except:
            pass

    if controlador:
        try:
            preco = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/strong').text
        except:
            pass

    dicio[num] = preco

dicio

{'09970.5308': 17000000.0,
 '09970.4672': 6000000.0,
 '09970.4130': 5500000.0,
 '09970.4786': 4000000.0,
 '09970.4321': 3700000.0,
 '01587.019': 3600000.0,
 '09970.4132': 3400000.0,
 '09970.4659': 3200000.0,
 '09970.4131': 3000000.0,
 '09970.4352': 2950000.0,
 '09970.4718': 2700000.0,
 '09970.3953': 2506726.41,
 '09970.4134': 2500000.0,
 '09970.4137': 2500000.0,
 '09970.4140': 2500000.0,
 '09970.4949': 2500000.0,
 '09970.4279': 2490000.0,
 '00138.014': 2450000.0,
 '09970.5305': 2320000.0,
 '09970.4185': 2300000.0,
 '09970.5131': 2200000.0,
 '09970.4158': 2000000.0,
 '09970.4160': 2000000.0,
 '09970.4157': 2000000.0,
 '09970.4999': 1990000.0,
 '09970.5025': 1950000.0,
 '09970.4742': 1800000.0,
 '09970.5243': 1800000.0,
 '09970.5254': 1790000.0,
 '09970.4340': 1700000.0,
 '09970.4716': 1690000.0,
 '09970.4710': 1650000.0,
 '09970.4573': 1650000.0,
 '09970.5171': 1620000.0,
 '01410.001': 1600000.0,
 '09970.4245': 1600000.0,
 '09970.5011': 1600000.0,
 '09970.4597': 1590000.0,
 '09970.4849'